# Project 1
- **Team Members**: Chika Ozodiegwu, Kelsey Wyatt, Libardo Lambrano, Kurt Pessa

![](Images/florida_covid19_data.jpg)

### Data set used:: 
* https://open-fdoh.hub.arcgis.com/datasets/florida-covid19-case-line-data


In [6]:
import pandas as pd

df = pd.read_csv("Resources/Florida_COVID19_Case_Line_Data.csv")

df.head(3)

,ObjectId,County,Age,Age_group,Gender,Jurisdiction,Travel_related,Origin,EDvisit,Hospitalized,Died,Case_,Contact,Case1,EventDate,ChartDate
0,1,Pinellas,32.0,25-34 years,Male,FL resident,No,NaN,NO,NO,NaN,Yes,Yes,2020/03/18 05:00:00,2020/03/08 00:00:00,2020/03/18 05:00:00
1,2,Pinellas,20.0,15-24 years,Female,FL resident,Yes,FRANCE,NO,NO,NaN,Yes,NO,2020/06/23 05:00:00,2020/06/22 00:00:00,2020/06/23 05:00:00
2,3,Citrus,80.0,75-84 years,Male,FL resident,Yes,DE,YES,YES,Yes,Yes,Yes,2020/03/18 05:00:00,2020/03/14 00:00:00,2020/03/18 05:00:00


# Hospitalizations rate in Florida has changed since reopening?
### Research Question to Answer:
* “Has hospitalizations (#) in Florida changed since reopening?” 

### Part 1: Hypothesis Testing
* Divide Hospitalization data in two groups of data
    * Prior to opening
    * After opening
* Decide on the date:
    * May 4th - restaurants opening to 25% capacity
    * June  (Miami opening beaches)
    
#### Six Steps for Hypothesis Testing
1. Identify
    - Populations:
        * Prior to opening
        * After opening
    - Distribution:
        * Distribution
2. State the hypotheses
3. Characteristics of the comparison distribution
4. Critical values
5. Calculate
6. Decide

### Part 2: Visualization
* Trends

### Further Inquries
* Increases in groups?
* Age
* Gender
* Ethnicitiy

### Rough Breakdown of Tasks
* Data Massaging